In [14]:
# probably going to have to install this if you haven't already
from sportsreference.nfl.boxscore import Boxscores, Boxscore

# the usual imports
import pandas as pd
import numpy as np

# for models
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

In [79]:
# The game_Data function is extracting game statistics for each game. It loops through each game and each week grabbing the statistics referenced.
# This function creates the foundation for our final dataset.
def game_data(game_df,game_stats):
    try:
        # Creates a dataframe for the away_team and the home_team. Sets column names to be exact matches between the two.
        away_team_df = game_df[['away_name', 'away_abbr', 'away_score']].rename(columns = {'away_name': 'team_name', 'away_abbr': 'team_abbr', 'away_score': 'score'})
        home_team_df = game_df[['home_name','home_abbr', 'home_score']].rename(columns = {'home_name': 'team_name', 'home_abbr': 'team_abbr', 'home_score': 'score'})
        try:
            if game_df.loc[0,'away_score'] > game_df.loc[0,'home_score']:
                away_team_df = pd.merge(away_team_df, pd.DataFrame({'game_won' : [1], 'game_lost' : [0]}),left_index = True, right_index = True)
                home_team_df = pd.merge(home_team_df, pd.DataFrame({'game_won' : [0], 'game_lost' : [1]}),left_index = True, right_index = True)
            elif game_df.loc[0,'away_score'] < game_df.loc[0,'home_score']:
                away_team_df = pd.merge(away_team_df, pd.DataFrame({'game_won' : [0], 'game_lost' : [1]}),left_index = True, right_index = True)
                home_team_df = pd.merge(home_team_df, pd.DataFrame({'game_won' : [1], 'game_lost' : [0]}),left_index = True, right_index = True)
            else: 
                away_team_df = pd.merge(away_team_df, pd.DataFrame({'game_won' : [0], 'game_lost' : [0]}),left_index = True, right_index = True)
                home_team_df = pd.merge(home_team_df, pd.DataFrame({'game_won' : [0], 'game_lost' : [0]}),left_index = True, right_index = True)
        except TypeError:
                away_team_df = pd.merge(away_team_df, pd.DataFrame({'game_won' : [np.nan], 'game_lost' : [np.nan]}),left_index = True, right_index = True)
                home_team_df = pd.merge(home_team_df, pd.DataFrame({'game_won' : [np.nan], 'game_lost' : [np.nan]}),left_index = True, right_index = True)        

        # Creating the away_team & home_team stats dataframe. Grabbing the selected stats and then renaming them to match home == away dataframe column names.
        away_stats_df = game_stats.dataframe[['away_first_downs', 'away_fourth_down_attempts',
               'away_fourth_down_conversions', 'away_fumbles', 'away_fumbles_lost',
               'away_interceptions', 'away_net_pass_yards', 'away_pass_attempts',
               'away_pass_completions', 'away_pass_touchdowns', 'away_pass_yards',
               'away_penalties', 'away_points', 'away_rush_attempts',
               'away_rush_touchdowns', 'away_rush_yards', 'away_third_down_attempts',
               'away_third_down_conversions', 'away_time_of_possession',
               'away_times_sacked', 'away_total_yards', 'away_turnovers',
               'away_yards_from_penalties', 'away_yards_lost_from_sacks']].reset_index().drop(columns ='index').rename(columns = {
               'away_first_downs': 'first_downs', 'away_fourth_down_attempts':'fourth_down_attempts',
               'away_fourth_down_conversions':'fourth_down_conversions' , 'away_fumbles': 'fumbles', 'away_fumbles_lost': 'fumbles_lost',
               'away_interceptions': 'interceptions', 'away_net_pass_yards':'net_pass_yards' , 'away_pass_attempts': 'pass_attempts',
               'away_pass_completions':'pass_completions' , 'away_pass_touchdowns': 'pass_touchdowns', 'away_pass_yards': 'pass_yards',
               'away_penalties': 'penalties', 'away_points': 'points', 'away_rush_attempts': 'rush_attempts',
               'away_rush_touchdowns': 'rush_touchdowns', 'away_rush_yards': 'rush_yards', 'away_third_down_attempts': 'third_down_attempts',
               'away_third_down_conversions': 'third_down_conversions', 'away_time_of_possession': 'time_of_possession',
               'away_times_sacked': 'times_sacked', 'away_total_yards': 'total_yards', 'away_turnovers': 'turnovers',
               'away_yards_from_penalties':'yards_from_penalties', 'away_yards_lost_from_sacks': 'yards_lost_from_sacks'})

        home_stats_df = game_stats.dataframe[['home_first_downs', 'home_fourth_down_attempts',
               'home_fourth_down_conversions', 'home_fumbles', 'home_fumbles_lost',
               'home_interceptions', 'home_net_pass_yards', 'home_pass_attempts',
               'home_pass_completions', 'home_pass_touchdowns', 'home_pass_yards',
               'home_penalties', 'home_points', 'home_rush_attempts',
               'home_rush_touchdowns', 'home_rush_yards', 'home_third_down_attempts',
               'home_third_down_conversions', 'home_time_of_possession',
               'home_times_sacked', 'home_total_yards', 'home_turnovers',
               'home_yards_from_penalties', 'home_yards_lost_from_sacks']].reset_index().drop(columns = 'index').rename(columns = {
               'home_first_downs': 'first_downs', 'home_fourth_down_attempts':'fourth_down_attempts',
               'home_fourth_down_conversions':'fourth_down_conversions' , 'home_fumbles': 'fumbles', 'home_fumbles_lost': 'fumbles_lost',
               'home_interceptions': 'interceptions', 'home_net_pass_yards':'net_pass_yards' , 'home_pass_attempts': 'pass_attempts',
               'home_pass_completions':'pass_completions' , 'home_pass_touchdowns': 'pass_touchdowns', 'home_pass_yards': 'pass_yards',
               'home_penalties': 'penalties', 'home_points': 'points', 'home_rush_attempts': 'rush_attempts',
               'home_rush_touchdowns': 'rush_touchdowns', 'home_rush_yards': 'rush_yards', 'home_third_down_attempts': 'third_down_attempts',
               'home_third_down_conversions': 'third_down_conversions', 'home_time_of_possession': 'time_of_possession',
               'home_times_sacked': 'times_sacked', 'home_total_yards': 'total_yards', 'home_turnovers': 'turnovers',
               'home_yards_from_penalties':'yards_from_penalties', 'home_yards_lost_from_sacks': 'yards_lost_from_sacks'})
        
        # Merge the team_df & stats_df for both home & away teams. Set the left_index & right_index to True so that both dataframes merge on the same indices. 
        away_team_df = pd.merge(away_team_df, away_stats_df,left_index = True, right_index = True)
        home_team_df = pd.merge(home_team_df, home_stats_df,left_index = True, right_index = True)
        try:
            # Converting time_of_possession from MM:SS format into seconds(int). 
            away_team_df['time_of_possession'] = (int(away_team_df['time_of_possession'].loc[0][0:2]) * 60) + int(away_team_df['time_of_possession'].loc[0][3:5])
            home_team_df['time_of_possession'] = (int(home_team_df['time_of_possession'].loc[0][0:2]) * 60) + int(home_team_df['time_of_possession'].loc[0][3:5])
        except TypeError:
            away_team_df['time_of_possession'] = np.nan
            home_team_df['time_of_possession'] = np.nan
    except TypeError:
        away_team_df = pd.DataFrame()
        home_team_df = pd.DataFrame()
    return away_team_df, home_team_df

def game_data_up_to_week(weeks,year):
    weeks_games_df = pd.DataFrame()
    for w in range(len(weeks)):
        date_string = str(weeks[w]) + '-' + str(year)
        week_scores = Boxscores(weeks[w],year)
        week_games_df = pd.DataFrame()
        for g in range(len(week_scores.games[date_string])):
            game_str = week_scores.games[date_string][g]['boxscore']
            game_stats = Boxscore(game_str)
            game_df = pd.DataFrame(week_scores.games[date_string][g], index = [0])
            away_team_df, home_team_df = game_data(game_df,game_stats)
            away_team_df['week'] = weeks[w]
            home_team_df['week'] = weeks[w]
            week_games_df = pd.concat([week_games_df,away_team_df])
            week_games_df = pd.concat([week_games_df,home_team_df])
        weeks_games_df = pd.concat([weeks_games_df,week_games_df])
    return weeks_games_df

def get_schedule(year):
    weeks = list(range(1,18))
    schedule_df = pd.DataFrame()
    for w in range(len(weeks)):
        date_string = str(weeks[w]) + '-' + str(year)
        week_scores = Boxscores(weeks[w],year)
        week_games_df = pd.DataFrame()
        for g in range(len(week_scores.games[date_string])):
            game = pd.DataFrame(week_scores.games[date_string][g], index = [0])[['away_name', 'away_abbr','home_name', 'home_abbr','winning_name', 'winning_abbr' ]]
            game['week'] = weeks[w]
            week_games_df = pd.concat([week_games_df,game])
        schedule_df = pd.concat([schedule_df, week_games_df]).reset_index().drop(columns = 'index') 
    return schedule_df 

def agg_weekly_data(schedule_df,weeks_games_df,current_week,weeks):
    schedule_df = schedule_df[schedule_df.week < current_week]
    agg_games_df = pd.DataFrame()
    for w in range(1,len(weeks)):
        games_df = schedule_df[schedule_df.week == weeks[w]]
        agg_weekly_df = weeks_games_df[weeks_games_df.week < weeks[w]].drop(columns = ['score','week','game_won', 'game_lost']).groupby(by=["team_name", "team_abbr"]).mean().reset_index()
        win_loss_df = weeks_games_df[weeks_games_df.week < weeks[w]][["team_name", "team_abbr",'game_won', 'game_lost']].groupby(by=["team_name", "team_abbr"]).sum().reset_index()
        win_loss_df['win_perc'] = win_loss_df['game_won'] / (win_loss_df['game_won'] + win_loss_df['game_lost'])
        win_loss_df = win_loss_df.drop(columns = ['game_won', 'game_lost'])

        try:
            agg_weekly_df['fourth_down_perc'] = agg_weekly_df['fourth_down_conversions'] / agg_weekly_df['fourth_down_attempts']  
        except ZeroDivisionError:
            agg_weekly_df['fourth_down_perc'] = 0 
        agg_weekly_df['fourth_down_perc'] = agg_weekly_df['fourth_down_perc'].fillna(0)

        try:
            agg_weekly_df['third_down_perc'] = agg_weekly_df['third_down_conversions'] / agg_weekly_df['third_down_attempts']  
        except ZeroDivisionError:
            agg_weekly_df['third_down_perc'] = 0
        agg_weekly_df['third_down_perc'] = agg_weekly_df['third_down_perc'].fillna(0)  

        agg_weekly_df = agg_weekly_df.drop(columns = ['fourth_down_attempts', 'fourth_down_conversions', 'third_down_attempts', 'third_down_conversions'])
        agg_weekly_df = pd.merge(win_loss_df,agg_weekly_df,left_on = ['team_name', 'team_abbr'], right_on = ['team_name', 'team_abbr'])

        away_df = pd.merge(games_df,agg_weekly_df,how = 'inner', left_on = ['away_name', 'away_abbr'], right_on = ['team_name', 'team_abbr']).drop(columns = ['team_name', 'team_abbr']).rename(columns = {
                'win_perc': 'away_win_perc',
               'first_downs': 'away_first_downs', 'fumbles': 'away_fumbles', 'fumbles_lost':'away_fumbles_lost', 'interceptions':'away_interceptions',
               'net_pass_yards': 'away_net_pass_yards', 'pass_attempts':'away_pass_attempts', 'pass_completions':'away_pass_completions',
               'pass_touchdowns':'away_pass_touchdowns', 'pass_yards':'away_pass_yards', 'penalties':'away_penalties', 'points':'away_points', 'rush_attempts':'away_rush_attempts',
               'rush_touchdowns':'away_rush_touchdowns', 'rush_yards':'away_rush_yards', 'time_of_possession':'away_time_of_possession', 'times_sacked':'away_times_sacked',
               'total_yards':'away_total_yards', 'turnovers':'away_turnovers', 'yards_from_penalties':'away_yards_from_penalties',
               'yards_lost_from_sacks': 'away_yards_lost_from_sacks', 'fourth_down_perc':'away_fourth_down_perc', 'third_down_perc':'away_third_down_perc'})

        home_df = pd.merge(games_df,agg_weekly_df,how = 'inner', left_on = ['home_name', 'home_abbr'], right_on = ['team_name', 'team_abbr']).drop(columns = ['team_name', 'team_abbr']).rename(columns = {
                'win_perc': 'home_win_perc',
               'first_downs': 'home_first_downs', 'fumbles': 'home_fumbles', 'fumbles_lost':'home_fumbles_lost', 'interceptions':'home_interceptions',
               'net_pass_yards': 'home_net_pass_yards', 'pass_attempts':'home_pass_attempts', 'pass_completions':'home_pass_completions',
               'pass_touchdowns':'home_pass_touchdowns', 'pass_yards':'home_pass_yards', 'penalties':'home_penalties', 'points':'home_points', 'rush_attempts':'home_rush_attempts',
               'rush_touchdowns':'home_rush_touchdowns', 'rush_yards':'home_rush_yards', 'time_of_possession':'home_time_of_possession', 'times_sacked':'home_times_sacked',
               'total_yards':'home_total_yards', 'turnovers':'home_turnovers', 'yards_from_penalties':'home_yards_from_penalties',
               'yards_lost_from_sacks': 'home_yards_lost_from_sacks', 'fourth_down_perc':'home_fourth_down_perc', 'third_down_perc':'home_third_down_perc'})

        agg_weekly_df = pd.merge(away_df,home_df,left_on = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'winning_name',
               'winning_abbr', 'week'], right_on = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'winning_name',
               'winning_abbr', 'week'])

        agg_weekly_df['win_perc_dif'] = agg_weekly_df['away_win_perc'] - agg_weekly_df['home_win_perc']
        agg_weekly_df['first_downs_dif'] = agg_weekly_df['away_first_downs'] - agg_weekly_df['home_first_downs']
        agg_weekly_df['fumbles_dif'] = agg_weekly_df['away_fumbles'] - agg_weekly_df['home_fumbles']
        agg_weekly_df['interceptions_dif'] = agg_weekly_df['away_interceptions'] - agg_weekly_df['home_interceptions']
        agg_weekly_df['net_pass_yards_dif'] = agg_weekly_df['away_net_pass_yards'] - agg_weekly_df['home_net_pass_yards']
        agg_weekly_df['pass_attempts_dif'] = agg_weekly_df['away_pass_attempts'] - agg_weekly_df['home_pass_attempts']
        agg_weekly_df['pass_completions_dif'] = agg_weekly_df['away_pass_completions'] - agg_weekly_df['home_pass_completions']
        agg_weekly_df['pass_touchdowns_dif'] = agg_weekly_df['away_pass_touchdowns'] - agg_weekly_df['home_pass_touchdowns']
        agg_weekly_df['pass_yards_dif'] = agg_weekly_df['away_pass_yards'] - agg_weekly_df['home_pass_yards']
        agg_weekly_df['penalties_dif'] = agg_weekly_df['away_penalties'] - agg_weekly_df['home_penalties']
        agg_weekly_df['points_dif'] = agg_weekly_df['away_points'] - agg_weekly_df['home_points']
        agg_weekly_df['rush_attempts_dif'] = agg_weekly_df['away_rush_attempts'] - agg_weekly_df['home_rush_attempts']
        agg_weekly_df['rush_touchdowns_dif'] = agg_weekly_df['away_rush_touchdowns'] - agg_weekly_df['home_rush_touchdowns']
        agg_weekly_df['rush_yards_dif'] = agg_weekly_df['away_rush_yards'] - agg_weekly_df['home_rush_yards']
        agg_weekly_df['time_of_possession_dif'] = agg_weekly_df['away_time_of_possession'] - agg_weekly_df['home_time_of_possession']
        agg_weekly_df['times_sacked_dif'] = agg_weekly_df['away_times_sacked'] - agg_weekly_df['home_times_sacked']
        agg_weekly_df['total_yards_dif'] = agg_weekly_df['away_total_yards'] - agg_weekly_df['home_total_yards']
        agg_weekly_df['turnovers_dif'] = agg_weekly_df['away_turnovers'] - agg_weekly_df['home_turnovers']
        agg_weekly_df['yards_from_penalties_dif'] = agg_weekly_df['away_yards_from_penalties'] - agg_weekly_df['home_yards_from_penalties']
        agg_weekly_df['yards_lost_from_sacks_dif'] = agg_weekly_df['away_yards_lost_from_sacks'] - agg_weekly_df['home_yards_lost_from_sacks']
        agg_weekly_df['fourth_down_perc_dif'] = agg_weekly_df['away_fourth_down_perc'] - agg_weekly_df['home_fourth_down_perc']
        agg_weekly_df['third_down_perc_dif'] = agg_weekly_df['away_third_down_perc'] - agg_weekly_df['home_third_down_perc']

        agg_weekly_df = agg_weekly_df.drop(columns = ['away_win_perc',
               'away_first_downs', 'away_fumbles', 'away_fumbles_lost', 'away_interceptions',
               'away_net_pass_yards', 'away_pass_attempts','away_pass_completions',
               'away_pass_touchdowns', 'away_pass_yards', 'away_penalties', 'away_points', 'away_rush_attempts',
               'away_rush_touchdowns', 'away_rush_yards', 'away_time_of_possession', 'away_times_sacked',
               'away_total_yards', 'away_turnovers', 'away_yards_from_penalties',
               'away_yards_lost_from_sacks','away_fourth_down_perc', 'away_third_down_perc','home_win_perc',
               'home_first_downs', 'home_fumbles', 'home_fumbles_lost', 'home_interceptions',
               'home_net_pass_yards', 'home_pass_attempts','home_pass_completions',
               'home_pass_touchdowns', 'home_pass_yards', 'home_penalties', 'home_points', 'home_rush_attempts',
               'home_rush_touchdowns', 'home_rush_yards', 'home_time_of_possession', 'home_times_sacked',
               'home_total_yards', 'home_turnovers', 'home_yards_from_penalties',
               'home_yards_lost_from_sacks','home_fourth_down_perc', 'home_third_down_perc'])
        
        if (weeks[w] == current_week and weeks[w] > 3 and agg_weekly_df['winning_name'].isnull().values.any()):
            agg_weekly_df['result'] = np.nan
            print(f"Week {weeks[w]} games have not finished yet.")
        else:
            agg_weekly_df['result'] = agg_weekly_df['winning_name'] == agg_weekly_df['away_name']
            agg_weekly_df['result'] = agg_weekly_df['result'].astype('float')
        agg_weekly_df = agg_weekly_df.drop(columns = ['winning_name', 'winning_abbr'])
        agg_games_df = pd.concat([agg_games_df, agg_weekly_df])
    agg_games_df = agg_games_df.reset_index().drop(columns = 'index')
    # What is .drop(index = 20) doing?
    agg_games_df = agg_games_df.drop(index = 20, axis=0)
    return agg_games_df

def get_elo():
    elo_df = pd.read_csv('https://projects.fivethirtyeight.com/nfl-api/nfl_elo_latest.csv')
    elo_df = elo_df.drop(columns = ['season','neutral' ,'playoff', 'elo_prob1', 'elo_prob2', 'elo1_post', 'elo2_post',
           'qbelo1_pre', 'qbelo2_pre', 'qb1', 'qb2', 'qb1_adj', 'qb2_adj', 'qbelo_prob1', 'qbelo_prob2',
           'qb1_game_value', 'qb2_game_value', 'qb1_value_post', 'qb2_value_post',
           'qbelo1_post', 'qbelo2_post', 'score1', 'score2', 'quality', 'importance', 'total_rating'])
    elo_df.date = pd.to_datetime(elo_df.date)
    elo_df = elo_df[elo_df.date < '01-05-2022']

    elo_df['team1'] = elo_df['team1'].replace(['KC', 'JAX', 'CAR', 'BAL', 'BUF', 'MIN', 'DET', 'ATL', 'NE', 'WSH',
           'CIN', 'NO', 'SF', 'LAR', 'NYG', 'DEN', 'CLE', 'IND', 'TEN', 'NYJ',
           'TB', 'MIA', 'PIT', 'PHI', 'GB', 'CHI', 'DAL', 'ARI', 'LAC', 'HOU',
           'SEA', 'OAK'],
            ['kan','jax','car', 'rav', 'buf', 'min', 'det', 'atl', 'nwe', 'was', 
            'cin', 'nor', 'sfo', 'ram', 'nyg', 'den', 'cle', 'clt', 'oti', 'nyj', 
             'tam','mia', 'pit', 'phi', 'gnb', 'chi', 'dal', 'crd', 'sdg', 'htx', 'sea', 'rai' ])
    elo_df['team2'] = elo_df['team2'].replace(['KC', 'JAX', 'CAR', 'BAL', 'BUF', 'MIN', 'DET', 'ATL', 'NE', 'WSH',
           'CIN', 'NO', 'SF', 'LAR', 'NYG', 'DEN', 'CLE', 'IND', 'TEN', 'NYJ',
           'TB', 'MIA', 'PIT', 'PHI', 'GB', 'CHI', 'DAL', 'ARI', 'LAC', 'HOU',
           'SEA', 'OAK'],
            ['kan','jax','car', 'rav', 'buf', 'min', 'det', 'atl', 'nwe', 'was', 
            'cin', 'nor', 'sfo', 'ram', 'nyg', 'den', 'cle', 'clt', 'oti', 'nyj', 
             'tam','mia', 'pit', 'phi', 'gnb', 'chi', 'dal', 'crd', 'sdg', 'htx', 'sea', 'rai' ])
    return elo_df

def merge_rankings(agg_games_df,elo_df):
    agg_games_df = pd.merge(agg_games_df, elo_df, how = 'inner', left_on = ['home_abbr', 'away_abbr'], right_on = ['team1', 'team2']).drop(columns = ['date','team1', 'team2'])
    agg_games_df['elo_dif'] = agg_games_df['elo2_pre'] - agg_games_df['elo1_pre']
    agg_games_df['qb_dif'] = agg_games_df['qb2_value_pre'] - agg_games_df['qb1_value_pre']
    agg_games_df = agg_games_df.drop(columns = ['elo1_pre', 'elo2_pre', 'qb1_value_pre', 'qb2_value_pre'])
    return agg_games_df

def prep_test_train(current_week,weeks,year):
    current_week = current_week + 1
    schedule_df  = get_schedule(year)
    weeks_games_df = game_data_up_to_week(weeks,year)
    agg_games_df = agg_weekly_data(schedule_df,weeks_games_df,current_week,weeks)
    elo_df = get_elo()
    agg_games_df = merge_rankings(agg_games_df, elo_df)
    train_df = agg_games_df[agg_games_df.result.notna()]
    current_week = current_week - 1
    test_df = agg_games_df[agg_games_df.week == current_week]
    return test_df, train_df

def display(y_pred,X_test):
    for g in range(len(y_pred)):
        #win_prob = np.round(y_pred[g],2)
        win_prob = int(y_pred[g] * 100)
        away_team = X_test.reset_index().drop(columns = 'index').loc[g,'away_name']
        home_team = X_test.reset_index().drop(columns = 'index').loc[g,'home_name']
        print(f'The {away_team} have a probability of {win_prob}% of beating the {home_team}.')
        
# Create dataframe to hold prediction results
def create_predictions_df(current_week, y_pred_df):
    results_df = pred_games_df[['week','away_name','home_name']]
    results_df['away_team_win_%'] = y_pred_df
    results_df['home_team_win_%'] = 1 - results_df['away_team_win_%']
    results_df['predicted_winner'] = 0

    # Loop through results dataframe and add predicted_winner name to column
    for index, row in results_df.iterrows():
        if results_df.loc[index,'away_team_win_%'] > results_df.loc[index,'home_team_win_%']:
            results_df.loc[index,'predicted_winner'] = results_df.loc[index,'away_name']
        else:
            results_df.loc[index,'predicted_winner'] = results_df.loc[index,'home_name']

    # Set Index to the 'week' column
    results_df=results_df.set_index('week')      

    # Use get_schedule function to pull dataframe that shows actual weekly winners. Includes all weeks.
    weekly_winner = get_schedule('2021')
    weekly_winner = weekly_winner.drop(columns=['away_abbr','home_abbr','winning_abbr'])
    weekly_winner = weekly_winner.rename(columns={'away_name':'ww_away_name','home_name':'ww_home_name'})

    # Slices weekly_winner dataframe based on current week model is predicting.
    current_week_wins = weekly_winner.loc[weekly_winner['week'] == current_week]
    current_week_wins = current_week_wins.set_index('week')

    # Sort both dataframes by away_name
    results_df = results_df.sort_values(by='away_name')
    current_week_wins = current_week_wins.sort_values(by='ww_away_name')

    # Combine results & weekly_winner df's. Drop dupe columns and rename winning_name to 'actual_winner'
    actual_results_df = (pd.concat([results_df,current_week_wins],axis=1)).drop(columns=['ww_away_name','ww_home_name']).rename(columns={'winning_name':'actual_winner'})
    return actual_results_df

## Get all the data for 2021, up to the current week 13

In [16]:
# this step takes about five minutes to run 
current_week = 13
weeks = list(range(1,current_week + 1))
year = 2021

pred_games_df, comp_games_df = prep_test_train(current_week,weeks,year)

In [45]:
comp_games_df = pd.concat([comp_games_df, pred_games_df])

In [56]:
comp_games_df.tail(30)

,away_name,away_abbr,home_name,home_abbr,week,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
161,Cleveland Browns,cle,Baltimore Ravens,rav,12,-0.154545,-3.772727,-0.027273,-0.354545,-45.963636,-6.236364,-4.272727,-0.400000,-47.590909,1.472727,-2.518182,-1.690909,0.345455,5.818182,-187.427273,-0.581818,-40.145455,-0.300000,15.800000,-1.627273,-0.257576,0.020200,0.0,-51.409517,-103.082625
162,Seattle Seahawks,sea,Washington Football Team,was,12,-0.100000,-4.100000,-0.700000,-0.500000,-24.700000,-4.800000,-3.100000,0.000000,-13.700000,-0.500000,-1.800000,-5.000000,0.100000,-24.500000,-336.400000,1.000000,-49.200000,-0.900000,-9.300000,11.000000,-0.355072,-0.068234,0.0,26.031709,32.264597
163,Dallas Cowboys,dal,New Orleans Saints,nor,13,0.181818,4.636364,0.727273,0.090909,94.181818,7.181818,8.181818,0.090909,93.272727,2.363636,6.272727,-0.454545,0.090909,16.272727,86.181818,0.090909,110.454545,0.272727,25.636364,-0.909091,-0.062745,0.043014,1.0,-39.950244,159.962301
164,Arizona Cardinals,crd,Chicago Bears,chi,13,0.454545,4.090909,0.636364,-0.181818,86.090909,4.181818,6.727273,1.090909,80.000000,-0.090909,11.909091,2.000000,0.727273,-3.272727,166.909091,-1.000000,82.818182,-0.363636,-0.909091,-6.090909,0.335165,0.100212,1.0,171.867039,93.922882
165,Tampa Bay Buccaneers,tam,Atlanta Falcons,atl,13,0.272727,5.545455,-0.181818,-0.454545,80.454545,6.727273,5.272727,1.272727,75.818182,0.909091,13.454545,-1.181818,0.545455,10.818182,118.909091,-0.545455,91.272727,-0.272727,15.363636,-4.636364,0.188312,0.079343,1.0,237.310235,103.270532
166,Philadelphia Eagles,phi,New York Jets,nyj,13,0.143939,0.863636,0.250000,-1.060606,-52.015152,-9.045455,-6.068182,-0.280303,-63.257576,-0.242424,7.242424,8.825758,0.772727,71.916667,-49.515152,-1.068182,19.901515,-1.098485,-2.174242,-11.242424,-0.180952,0.049692,1.0,145.251604,103.227714
167,New York Giants,nyg,Miami Dolphins,mia,13,-0.053030,0.090909,-0.469697,-0.015152,-4.030303,-2.878788,-2.772727,-0.250000,-6.939394,-0.848485,-1.136364,-0.196970,-0.113636,10.651515,-63.128788,-0.325758,6.621212,-0.393939,-4.636364,-2.909091,-0.129412,-0.058413,0.0,-57.912337,-37.978877
168,Denver Broncos,den,Kansas City Chiefs,kan,13,-0.090909,-5.454545,-1.000000,-0.363636,-66.727273,-9.000000,-5.181818,-0.909091,-58.272727,-1.272727,-4.818182,0.909091,0.090909,2.818182,32.272727,1.000000,-63.909091,-0.909091,-9.000000,8.454545,0.030303,-0.138340,0.0,-173.767258,-78.641705
169,Indianapolis Colts,clt,Houston Texans,htx,13,0.318182,7.454545,-0.030303,-0.590909,35.287879,1.946970,0.454545,0.659091,27.492424,-2.159091,13.424242,3.734848,0.871212,66.227273,231.007576,-1.068182,101.515152,-0.386364,-13.219697,-7.795455,0.039683,0.027572,1.0,231.468424,37.359951
170,Minnesota Vikings,min,Detroit Lions,det,13,0.454545,1.909091,0.272727,-0.454545,72.090909,2.363636,1.818182,1.181818,61.272727,-0.090909,9.727273,3.090909,-0.272727,3.727273,85.363636,-1.181818,75.818182,-0.545455,8.636364,-10.818182,0.208075,0.054865,0.0,239.154443,100.393607


In [60]:
# Write out 2021 info, so I don't have to load it all the time
df = pd.concat([comp_games_df, pred_games_df], axis=0).drop_duplicates()
df.to_csv("2021_week_2_through_13.csv", index = False)

In [61]:
# read the full dataframe in and drop an unneeded column
df = pd.read_csv('2021_week_2_through_13.csv')

# Logistic Regression

### Logistic Regression Predictions

In [62]:
pred_week = 13
comp_games_df = df[df['week'] < pred_week]
pred_games_df = df[df['week'] == pred_week]

In [63]:
current_week = pred_week

In [64]:
train_df = comp_games_df
test_df = pred_games_df

X_train = train_df.drop(columns = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'week','result'])
y_train = train_df[['result']] 
X_test = test_df.drop(columns = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'week','result'])
y_test = test_df[['result']]

In [65]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [83]:
# Logistic Model
clf_unscaled = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf_scaled = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf_unscaled.fit(X_train, np.ravel(y_train.values))
clf_scaled.fit(X_train_scaled, np.ravel(y_train.values))

y_pred_unscaled = clf_unscaled.predict_proba(X_test)
y_pred_scaled = clf_scaled.predict_proba(X_test_scaled)

y_pred_unscaled = y_pred_unscaled[:,1]
y_pred_scaled = y_pred_scaled[:,1]

y_pred_unscaled_df = pd.DataFrame(y_pred_unscaled)
y_pred_scaled_df = pd.DataFrame(y_pred_scaled)

In [84]:
#Create a output dataframe for the lr scaled data
y_pred_scaled_predictions = create_predictions_df(current_week, y_pred_scaled)

y_pred_scaled_predictions

C:\Users\14065\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\14065\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:261: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\14065\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,away_name,home_name,away_team_win_%,home_team_win_%,predicted_winner,actual_winner
week,,,,,,
13,Arizona Cardinals,Chicago Bears,0.719638,0.280362,Arizona Cardinals,Arizona Cardinals
13,Baltimore Ravens,Pittsburgh Steelers,0.798032,0.201968,Baltimore Ravens,Pittsburgh Steelers
13,Dallas Cowboys,New Orleans Saints,0.679309,0.320691,Dallas Cowboys,Dallas Cowboys
13,Denver Broncos,Kansas City Chiefs,0.221579,0.778421,Kansas City Chiefs,Kansas City Chiefs
13,Indianapolis Colts,Houston Texans,0.859180,0.140820,Indianapolis Colts,Indianapolis Colts
13,Jacksonville Jaguars,Los Angeles Rams,0.284466,0.715534,Los Angeles Rams,Los Angeles Rams
13,Los Angeles Chargers,Cincinnati Bengals,0.384153,0.615847,Cincinnati Bengals,Los Angeles Chargers
13,Minnesota Vikings,Detroit Lions,0.695403,0.304597,Minnesota Vikings,Detroit Lions
13,New England Patriots,Buffalo Bills,0.458319,0.541681,Buffalo Bills,None


In [85]:
#Create a output dataframe for the lr scaled data
y_pred_unscaled_predictions = create_predictions_df(current_week, y_pred_unscaled)

y_pred_unscaled_predictions

C:\Users\14065\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\14065\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:261: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\14065\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,away_name,home_name,away_team_win_%,home_team_win_%,predicted_winner,actual_winner
week,,,,,,
13,Arizona Cardinals,Chicago Bears,0.721316,0.278684,Arizona Cardinals,Arizona Cardinals
13,Baltimore Ravens,Pittsburgh Steelers,0.797587,0.202413,Baltimore Ravens,Pittsburgh Steelers
13,Dallas Cowboys,New Orleans Saints,0.774261,0.225739,Dallas Cowboys,Dallas Cowboys
13,Denver Broncos,Kansas City Chiefs,0.300011,0.699989,Kansas City Chiefs,Kansas City Chiefs
13,Indianapolis Colts,Houston Texans,0.893573,0.106427,Indianapolis Colts,Indianapolis Colts
13,Jacksonville Jaguars,Los Angeles Rams,0.252504,0.747496,Los Angeles Rams,Los Angeles Rams
13,Los Angeles Chargers,Cincinnati Bengals,0.248399,0.751601,Cincinnati Bengals,Los Angeles Chargers
13,Minnesota Vikings,Detroit Lions,0.650606,0.349394,Minnesota Vikings,Detroit Lions
13,New England Patriots,Buffalo Bills,0.422850,0.577150,Buffalo Bills,None


### Get Logistic Regression Accuracy

In [10]:
# make a for loop to the get accuracy for each week
def accuracy_score_log_reg(df):
    
    accuracy = pd.DataFrame(columns=['week', 'Logistic Regression Accuracy - Unscaled', 'Logistic Regeression Accuracy - Scaled'])
    
    for w in df['week'].unique()[1:-1]:
        train_df = df[df['week'] < w]
        test_df = df[df['week'] == w]
        
        X_train = train_df.drop(columns = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'week','result'])
        y_train = train_df[['result']] 
        X_test = test_df.drop(columns = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'week','result'])
        y_test = test_df[['result']]
        
        # Create a StandardScaler instance
        scaler = StandardScaler()

        # Fit the scaler to the features training dataset
        X_scaler = scaler.fit(X_train)

        # Fit the scaler to the features training dataset
        X_train_scaled = X_scaler.transform(X_train)
        X_test_scaled = X_scaler.transform(X_test)
        
        clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

        clf.fit(X_train, np.ravel(y_train.values))
        clf.fit(X_train_scaled, np.ravel(y_train.values))
        
        y_pred_unscaled = clf.predict_proba(X_test)
        y_pred_scaled = clf.predict_proba(X_test_scaled)
        
        y_pred_unscaled = y_pred_unscaled[:,1]
        y_pred_scaled = y_pred_scaled[:,1]
        
        accuracy_score_unscaled = accuracy_score(y_test,np.round(y_pred_unscaled))
        accuracy_score_scaled = accuracy_score(y_test,np.round(y_pred_scaled))
        
        accuracy.loc[w,:] = [w, accuracy_score_unscaled, accuracy_score_scaled]
        
    return(accuracy)

In [11]:
logistic_regression_accuracy = accuracy_score_log_reg(df)
logistic_regression_accuracy

,week,Logistic Regression Accuracy - Unscaled,Logistic Regeression Accuracy - Scaled
3,3,0.533333,0.466667
4,4,0.5625,0.6875
5,5,0.75,0.625
6,6,0.785714,0.857143
7,7,0.769231,0.769231
8,8,0.533333,0.466667
9,9,0.642857,0.714286
11,11,0.533333,0.733333
12,12,0.533333,0.666667


# Tensorflow 

### Tensorflow Predictions

In [12]:
# load a previously built model.
nn = tf.keras.models.load_model("Resources/model_nn_2021_1_12_not_scaled_relu_relu_sigmoid_loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']_1000 epochs.h5")

OSError: No file or directory found at Resources/model_nn_2021_1_12_not_scaled_relu_relu_sigmoid_loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']_1000 epochs.h5

In [ ]:
pred_week = 13
comp_games_df = df[df['week'] < pred_week]
pred_games_df = df[df['week'] == pred_week]

In [ ]:
train_df = comp_games_df
test_df = pred_games_df

X_train = train_df.drop(columns = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'week','result'])
y_train = train_df[['result']] 
X_test = test_df.drop(columns = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'week','result'])
y_test = test_df[['result']]

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data        
nn_predictions_unscaled = nn.predict(X_test)       
nn_predictions_scaled = nn.predict(X_test_scaled)

print("Tensorflow - Unscaled \n")
display(nn_predictions_unscaled.squeeze(),test_df)

print("\nTensorflow - Scaled \n")
display(nn_predictions_scaled.squeeze(),test_df)

## Tensorflow Accuracy

In [ ]:
# make a for loop to the get accuracy for each week
def accuracy_score_tensorflow(df):
    
    accuracy = pd.DataFrame(columns=['week', 'Tensorflow Accuracy - Unscaled', 'Tensorflow Accuracy - Scaled'])
    
    for w in df['week'].unique()[1:-1]:
        train_df = df[df['week'] < w]
        test_df = df[df['week'] == w]
        
        X_train = train_df.drop(columns = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'week','result'])
        y_train = train_df[['result']] 
        X_test = test_df.drop(columns = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'week','result'])
        y_test = test_df[['result']]
        
        # Create a StandardScaler instance
        scaler = StandardScaler()

        # Fit the scaler to the features training dataset
        X_scaler = scaler.fit(X_train)

        # Fit the scaler to the features training dataset
        X_train_scaled = X_scaler.transform(X_train)
        X_test_scaled = X_scaler.transform(X_test)
        
        # Evaluate the model loss and accuracy metrics using the evaluate method and the test data        
        model_accuracy_unscaled = nn.evaluate(X_test, y_test, verbose=0)       
        model_accuracy_scaled = nn.evaluate(X_test_scaled, y_test, verbose=0)
    
        # Assign the accuracy to 
        accuracy.loc[w,:] = [w, model_accuracy_unscaled[1], model_accuracy_scaled[1]]
        
    return(accuracy)

In [ ]:
# Run the accuracy_score_Tensorflow function to get an array of accuracies
tensorflow_accuracy = accuracy_score_tensorflow_scaled(df)
tensorflow_accuracy

In [ ]:
pd.concat(
    [logistic_regression_accuracy, tensorflow_accuracy.drop(columns=['week'])], axis=1)

# Random Forest Model

In [ ]:
pred_week = 13
comp_games_df = df[df['week'] < pred_week]
pred_games_df = df[df['week'] == pred_week]

In [ ]:
train_df = comp_games_df
test_df = pred_games_df

X_train = train_df.drop(columns = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'week','result'])
y_train = train_df[['result']] 
X_test = test_df.drop(columns = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'week','result'])
y_test = test_df[['result']]

In [ ]:
# Create a random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

rf_model = RandomForestClassifier(n_estimators=100, random_state=1)

# Fitting the model
rf_fitted = rf_model.fit(X_train, np.ravel(y_train.values))

# Making predictions using the testing data
y_pred = rf_fitted.predict_proba(X_test)
y_pred = y_pred[:,0]

print("RandomForest - Unscaled \n")
display(y_pred, test_df)

In [ ]:
# make a for loop to the get accuracy for each week
def accuracy_score_random_forest(df):
    
    accuracy = pd.DataFrame(columns=['week', 'Random Forest Accuracy'])
    
    for w in df['week'].unique()[1:-1]:
        train_df = df[df['week'] < w]
        test_df = df[df['week'] == w]
        
        X_train = train_df.drop(columns = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'week','result'])
        y_train = train_df[['result']] 
        X_test = test_df.drop(columns = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'week','result'])
        y_test = test_df[['result']]
        
        # Fitting the model
        classifier = RandomForestClassifier(n_estimators=100, random_state=0)
        classifier.fit(X_train, np.ravel(y_train.values))
        y_pred = classifier.predict(X_test)
        
        model_accuracy = accuracy_score(y_test, y_pred)

        # Assign the accuracy to 
        accuracy.loc[w,:] = [w, model_accuracy]
        
    return(accuracy)

In [ ]:
random_forest_accuracy = accuracy_score_random_forest(df)
random_forest_accuracy

In [ ]:
print(pd.concat(
    [logistic_regression_accuracy, 
     tensorflow_accuracy.drop(columns=['week']),
     random_forest_accuracy.drop(columns=['week'])], 
    axis=1))